This Notebook would be used mainly for the developmet of the Capstones Project

In [3]:
import pandas as pd
import numpy as np

In [4]:
print ('Hello Capstone Project Course')

Hello Capstone Project Course


In [10]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_ac3f7afc30a549c58fb6cf226131b705 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='in2xZ9r7YRnAdOXHoMzjt_Qsh8HayHyJroyk42DYItDF',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_ac3f7afc30a549c58fb6cf226131b705.get_object(Bucket='capstoneproject-donotdelete-pr-f78ewbdz9fysl5',Key='CPC.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_0 = pd.read_excel(body)
df_data_0.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [22]:
df_data_0.replace('Not assigned',np.NaN);

In [34]:
df_data_0.dropna(inplace=True)
df_data_0.reset_index(inplace=True)
df_data_0.drop(['index'],axis=1,inplace=True)
df_data_0.head()
df_data_0.groupby(['Postal code'])
df_data_0.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
